# Chapter 13: Standardization and the Parametric G-Formula
This notebook goes through Chapter 13 of “Hernán MA, Robins JM (2019). Causal Inference. Boca Raton: Chapman & Hall/CRC, forthcoming”, which details the use of the parametric g-formula for causal inference using models. Within this notebook, I will use *zEpid* to recreate the analyses detailed in chapter 13. As an introduction to causal inference and the associated methods, I highly recommend reviewing this book, which the preprint is available for free at: https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/

### Data Preparation
Data comes from the National Health and Nutrition Examination Survey Data I Epidemiologic
Follow-up Study (NHEFS). The NHEFS was jointly initiated by the National Center for Health Statistics and the
National Institute on Aging in collaboration with other agencies of the United States Public Health Service. A
detailed description of the NHEFS, together with publicly available data sets and documentation, can be found at
wwwn.cdc.gov/nchs/nhanes/nhefs/ 

The data set used in the book and this tutorial is a subset of the full NHEFS. First, we will load the data and run some basic variable manipulations. We are interested in estimating the average causal effect ($E[Y^{a=1}] - E[Y^{a=0}]$) of stopping smoking (`qsmk`) on 10-year weight change (`wt82_71`). See Fine Point 12.2 for details on problems with this example (specifically, there is selection bias induced by how our treatment variable is defined)

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

from zepid.causal.gformula import TimeFixedGFormula

df = pd.read_csv('Data/nhefs.csv')
df.dropna(subset=['sex', 'age', 'race', 'wt82', 'ht', 
                  'school', 'alcoholpy', 'smokeintensity'], 
         inplace=True)

# recoding some variables
df['inactive'] = np.where(df['active'] == 2, 1, 0)
df['no_exercise'] = np.where(df['exercise'] == 2, 1, 0)
df['university'] = np.where(df['education'] == 5, 1, 0)

# creating quadratic terms
for col in ['age', 'wt71', 'smokeintensity', 'smokeyrs']:
    df[col+'_sq'] = df[col] * df[col]



## Section 13.3
To estimate the causal effect of quitting smoking on weight change (in kg), we will use the parametric g-formula. Specifically, we will assume $Y^a \amalg A|L$, where $L$ includes sex, race, age, education, smoking duration, smoking intensity, exercise, recreational activity, and baseline weight. We can express the g-formula as
$$ \hat{E}[Y^a] = \hat{E}\left[\hat{E}[Y|A=a,L]\right]$$
where the outer expectation marginalizes over the distribution of $L$

To estimate $\hat{E}[Y^a]$, we can use `TimeFixedGFormula` in *zEpid*. We will initialize `TimeFixedGFormula` with the data set (`df`), treatment (`qsmk`), and we will specify the optional argument `outcome_type` to model the outcome using linear regression. 

One advantage of the R approach to fitting models is that we can use `patsy` magic. Specifically, we will avoid creating indicator (dummy) variables for our categorical variables. Instead we will use `C(...)`, which tells patsy we want that variable to be a categorical variables. We will do this for `active` and `exercise`. Additionally, we can easily incorporate interactions between our variables (without having to manually code them). For an interaction between a column `A` and `B`, we would use `A:B` in the model statement

In [2]:
tfgf = TimeFixedGFormula(df, exposure='qsmk', outcome='wt82_71', outcome_type='normal')
tfgf.outcome_model('qsmk + sex + race + age + age_sq + C(education) + smokeintensity + ' + 
                   'smokeintensity_sq + smokeyrs + smokeyrs_sq + C(exercise) + ' + 
                   'C(active) + wt71 + wt71_sq', 
                   print_results=False)

# Estimating effect under treat-all
tfgf.fit(treatment='all')
rall = tfgf.marginal_outcome

# Estimating effect under treat-none
tfgf.fit(treatment='none')
nall = tfgf.marginal_outcome

In [4]:
# Bootstrapping confidence intervals
ests = []
for i in range(200):
    dfs = df.sample(n=df.shape[0], replace=True)
    tfgf = TimeFixedGFormula(dfs, exposure='qsmk', outcome='wt82_71', outcome_type='normal')
    tfgf.outcome_model('qsmk + sex + race + age + age_sq + C(education) + smokeintensity + ' + 
                       'smokeintensity_sq + smokeyrs + smokeyrs_sq + C(exercise) + ' + 
                       'C(active) + wt71 + wt71_sq', 
                       print_results=False)
    tfgf.fit(treatment='all')
    rall_b = tfgf.marginal_outcome
    tfgf.fit(treatment='none')
    nall_b = tfgf.marginal_outcome
    ests.append(rall_b - nall_b)

se = np.std(ests)
lcl = (rall - nall) - 1.96*se
ucl = (rall - nall) + 1.96*se

In [6]:
print('Average Causal Effect')
print('--------------------------------------')
print('E[Y|A=1] - E[Y|A=0] = ', np.round(rall - nall, 2))
print('95% CL: '+str(np.round(lcl, 2))+', '+str(np.round(ucl, 2)))
print('--------------------------------------')

Average Causal Effect
--------------------------------------
E[Y|A=1] - E[Y|A=0] =  3.46
95% CL: 2.56, 4.37
--------------------------------------


We obtain a similar result to the book. The confidence intervals differ slightly due to the random error of the bootstrap procedure.

## Conclusion
That concludes chapter 13 of "Causal Inference" by Hernan and Robins. Please review the other tutorials on this site for more details and features of `TimeFixedGFormula`. In the next tutorial, we will go through g-estimation of structural nested models